# Setup Libraries and Dataset


In [1]:
! git clone --recursive https://github.com/multi30k/dataset.git multi30k-dataset

Cloning into 'multi30k-dataset'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 313 (delta 17), reused 21 (delta 16), pack-reused 281
Receiving objects: 100% (313/313), 18.21 MiB | 4.50 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Submodule 'scripts/subword-nmt' (https://github.com/rsennrich/subword-nmt.git) registered for path 'scripts/subword-nmt'
Cloning into '/Users/venkatakrishnanvk/Desktop/ML & DL/Deep Learning/Pytorch-Tutorial/Natural Language Processing/multi30k-dataset/scripts/subword-nmt'...
remote: Enumerating objects: 597, done.        
remote: Counting objects: 100% (21/21), done.        
remote: Compressing objects: 100% (17/17), done.        
remote: Total 597 (delta 8), reused 12 (delta 4), pack-reused 576        
Receiving objects: 100% (597/597), 252.23 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (357/357), done.
Submodule path 'scripts/subword-nmt': checke

In [3]:
%%capture
# W and B -- For Logging
! pip install wandb

# Sacremoses -- For Tokenizing
! pip install sacremoses

# fairseq -- For training and evaluation of the model
# ! git clone https://github.com/pytorch/fairseq
# %cd fairseq
# ! pip install --editable ./
# %cd ..

In [4]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /Users/venkatakrishnanvk/.netrc


True

## Pre-process and Binarize to build Vocabularies

In [5]:
! fairseq-preprocess --source-lang de --target-lang en \
  --trainpref multi30k-dataset/data/task1/tok/train.lc.norm.tok \
  --validpref multi30k-dataset/data/task1/tok/val.lc.norm.tok \
  --testpref  multi30k-dataset/data/task1/tok/test_2018_flickr.lc.norm.tok \
  --destdir data-bin/multi30k.tokenized.de-en \
  --thresholdsrc 2 \
  --thresholdtgt 2

2023-03-19 09:31:13 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='fixed', scoring='bleu', task='translation', source_lang='de', target_lang='en', tr

# Train the model

In [7]:
! fairseq-train data-bin/multi30k.tokenized.de-en \
  --arch transformer \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --decoder-learned-pos \
  --max-epoch 10 \
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --wandb-project "Multi 30K En to De Translation"

2023-03-19 09:31:44 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'Multi 30K En to De Translation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}

# Evaluate the model

In [11]:
# ckpt_best, beam=5
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --wandb-project "Multi 30K En to De Translation"

2023-03-19 10:03:51 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'Multi 30K En to De Translation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasm

In [12]:
# ckpt_best, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

2023-03-19 10:06:12 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'Multi 30K En to De Translation', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasm

# Generate Average Checkpoint (Ensemble)

In [ ]:
! python fairseq/scripts/average_checkpoints.py --inputs checkpoints --num-epoch-checkpoints 5 --output checkpoints/averaged.pt

Namespace(checkpoint_upper_bound=None, inputs=['checkpoints'], num_epoch_checkpoints=5, num_update_checkpoints=None, output='checkpoints/averaged.pt')
averaging checkpoints:  ['checkpoints/checkpoint10.pt', 'checkpoints/checkpoint9.pt', 'checkpoints/checkpoint8.pt', 'checkpoints/checkpoint7.pt', 'checkpoints/checkpoint6.pt']
Finished writing averaged checkpoint to checkpoints/averaged.pt


In [ ]:
# ckpt_avg, beam=5
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

2020-11-25 05:46:45 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': 'Multi 30K En to De Translation', 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': True}, 'common_eval': {'_name': None, 'path': 'checkpoints/averaged.pt', 'post_process': None, 'quiet': True, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distribute

In [ ]:
# ckpt_avg, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

2020-11-25 06:18:29 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': 'Multi 30K En to De Translation', 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': True}, 'common_eval': {'_name': None, 'path': 'checkpoints/averaged.pt', 'post_process': None, 'quiet': True, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distribute

# Export the model

In [ ]:
! mkdir -p trained_model
! cp data-bin/multi30k.tokenized.de-en/dict.de.txt trained_model/dict.de.txt
! cp data-bin/multi30k.tokenized.de-en/dict.en.txt trained_model/dict.en.txt
! cp checkpoints/averaged.pt trained_model/model.pt

In [ ]:
! fairseq-interactive \
  --path trained_model/model.pt \
  --source-lang de --target-lang en \
  --tokenizer moses \
  --task translation --cpu \
  --beam 5 \
  trained_model/

2020-11-25 05:50:00 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': True, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': True}, 'common_eval': {'_name': None, 'path': 'trained_model/model.pt', 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'dis